In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import copy
from math import sqrt,ceil
from scipy.stats import linregress
import libraries as lib

In [27]:
hoofdmap = "../../../../OneDrive - HvA/Logfiles"

# Specificeer de belangrijke submappen
belangrijke_submappen = ['H2A', 'SOLAR']


In [30]:
data = []

for main_submap in belangrijke_submappen:
    main_submap_path = os.path.join(hoofdmap, main_submap)
    if not os.path.exists(main_submap_path):
        print(f"Waarschuwing: {main_submap_path} bestaat niet.")
        continue  # Sla deze submap over als deze niet bestaat
    
    for root, dirs, files in os.walk(main_submap_path):
        # Relative path ten opzichte van de hoofdmap
        relative_path = os.path.relpath(root, hoofdmap)
        # Splits de relative path in onderdelen
        path_parts = relative_path.split(os.sep) if relative_path != '.' else []
        
        # Identificeer de hoofdsubmap
        main_folder = path_parts[0] if path_parts else main_submap  # Veilig stellen
        
        # Voeg alleen bestanden toe
        for file_name in files:
            file_path = os.path.join(root, file_name)
            file_relative_path = os.path.relpath(file_path, hoofdmap)
            file_path_parts = file_relative_path.split(os.sep)
            
            # Verwijder de hoofdsubmap uit de path_parts
            if len(file_path_parts) > 1:
                sub_path_parts = file_path_parts[1:-1]  # Verwijder hoofdsubmap en bestandsnaam
            else:
                sub_path_parts = []
            
            data.append({
                'main_folder': main_folder,
                'pad': file_path,
                'path_parts': sub_path_parts,
                'filename': file_name
            })


In [31]:
# Maak een DataFrame van de verzamelde data
df = pd.DataFrame(data)

# Bepaal de maximale diepte van de mappenstructuur (exclusief de hoofdsubmap en bestandsnaam)
max_depth = df['path_parts'].apply(len).max()

# Beperk tot maximaal 6 niveaus zoals gewenst
max_levels = 6
max_depth = min(max_depth, max_levels)

# Maak kolomnamen aan voor elk niveau
kolomnamen = [f'map_level_{i+1}' for i in range(max_depth)]  # map_level_1, map_level_2, ..., map_level_6

# Splits de path_parts in aparte kolommen, vul aan met lege strings indien nodig
path_df = pd.DataFrame(df['path_parts'].tolist(), columns=kolomnamen)
for col in kolomnamen:
    if col not in path_df.columns:
        path_df[col] = ''

# Selecteer alleen de benodigde kolommen (max 6)
path_df = path_df.iloc[:, :max_levels]

# Combineer met de rest van de DataFrame en voeg de bestandsnaam toe
df_final = pd.concat([df[['main_folder', 'pad']], path_df, df['filename']], axis=1)

# Optioneel: Vul NaN waarden met lege strings voor betere leesbaarheid
df_final.fillna('', inplace=True)

# Herstructureer de kolom volgorde voor duidelijkheid
kolom_volgorde = ['main_folder', 'pad'] + kolomnamen + ['filename']
df_final = df_final[kolom_volgorde]

# Bekijk de DataFrame
df_final


,main_folder,pad,map_level_1,map_level_2,map_level_3,map_level_4,map_level_5,filename
0,H2A,../../../../OneDrive - HvA/Logfiles\H2A\2020\1...,2020,,,,,1_10-2020.zip
1,H2A,../../../../OneDrive - HvA/Logfiles\H2A\2021\2...,2021,20211202_nulmeting_h2a,0024,,,4.csv
2,H2A,../../../../OneDrive - HvA/Logfiles\H2A\2021\2...,2021,20211202_nulmeting_h2a,0024,,,F.csv
3,H2A,../../../../OneDrive - HvA/Logfiles\H2A\2021\2...,2021,20211202_nulmeting_h2a,0024,,,I.csv
4,H2A,../../../../OneDrive - HvA/Logfiles\H2A\2021\2...,2021,20211202_nulmeting_h2a,0024_new,,,4.csv
...,...,...,...,...,...,...,...,...
9132,SOLAR,../../../../OneDrive - HvA/Logfiles\SOLAR\2024...,2024,old,terugvaart_monaco-old,8-7-2024,0232,8.csv
9133,SOLAR,../../../../OneDrive - HvA/Logfiles\SOLAR\2024...,2024,old,terugvaart_monaco-old,8-7-2024,0232,cmlogger-status.log
9134,SOLAR,../../../../OneDrive - HvA/Logfiles\SOLAR\2024...,2024,old,terugvaart_monaco-old,8-7-2024,0232,F.csv
9135,SOLAR,../../../../OneDrive - HvA/Logfiles\SOLAR\2024...,2024,old,terugvaart_monaco-old,8-7-2024,0232,I.csv
